In [1]:
import pandas as pd
import plotly

In [2]:
# Download a data file from Yahoo.com
path = "/Users/ching/Downloads/AAPL.csv"
apple_df = pd.read_csv(path)
apple_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-19,141.690002,144.949997,141.500000,143.860001,143.011597,61758300
1,2022-10-20,143.020004,145.889999,142.649994,143.389999,142.544373,64522000
2,2022-10-21,142.869995,147.850006,142.649994,147.270004,146.401489,86548600
3,2022-10-24,147.190002,150.229996,146.000000,149.449997,148.568619,75981900
4,2022-10-25,150.089996,152.490005,149.360001,152.339996,151.441589,74732300
...,...,...,...,...,...,...,...
245,2023-10-11,178.199997,179.850006,177.600006,179.800003,179.800003,47551100
246,2023-10-12,180.070007,182.339996,179.039993,180.710007,180.710007,56743100
247,2023-10-13,181.419998,181.929993,178.139999,178.850006,178.850006,51427100
248,2023-10-16,176.750000,179.080002,176.509995,178.720001,178.720001,52517000


In [3]:
import requests
import pandas as pd
from io import StringIO


df = pd.DataFrame(apple_df)
# Convert the DataFrame to a CSV string
csv_string = df.to_csv(index=False)
# Define the FastAPI endpoint URL
upload_url = "http://localhost:8000/upload-dataframe/"  # Replace with your actual URL
# Create a file-like object from the CSV string
csv_file = StringIO(csv_string)
# Send a POST request to upload the DataFrame
files = {'file': ('data.csv', csv_file, 'text/csv')}
response = requests.post(upload_url, files=files)
# Print the response
print(response.text)


{"message":"DataFrame uploaded and saved."}


### get Dataframe

In [19]:
import requests

# Update with the desired file name
filename = "data.csv"  

# Make a GET request to the endpoint
response = requests.get(f"http://localhost:8000/get-dataframe/?filename={filename}")

# Check the response status code
if response.status_code == 200:
    data = response.json()
    data = data["dataframe"]
    # Now you can work with the DataFrame in your Jupyter Notebook
    
else:
    print("Request failed with status code:", response.status_code)

pd.DataFrame(data).head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-19,141.690002,144.949997,141.500000,143.860001,143.011597,61758300
1,2022-10-20,143.020004,145.889999,142.649994,143.389999,142.544373,64522000
2,2022-10-21,142.869995,147.850006,142.649994,147.270004,146.401489,86548600
3,2022-10-24,147.190002,150.229996,146.000000,149.449997,148.568619,75981900
4,2022-10-25,150.089996,152.490005,149.360001,152.339996,151.441589,74732300


### Return all models

In [5]:
import requests
from pprint import pprint

# Define the URL of your FastAPI application
url = "http://localhost:8000/model/get_all_models"  # Replace with the actual URL of your FastAPI application

# Send a GET request to the / endpoint
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    data = response.json()
    pprint(data)

else:
    print("Request failed with status code:", response.status_code)


[{'asset_name': 'Testing_asset_1',
  'currency': 'USD, JPY',
  'id': 1,
  'market': 'US',
  'model_name': 'Machine_learning_001'},
 {'asset_name': 'asset 2',
  'currency': 'USD',
  'id': 2,
  'market': 'US',
  'model_name': 'model 2'}]


### Add model to Database

In [8]:
import requests

# Define the URL of your FastAPI application
url = "http://localhost:8000/model/post_new_model"  # Replace with the actual URL of your FastAPI application
# Define the JSON data you want to send in the POST request
data = {
    "asset_name": "asset 1",
    "model_name": "model 1",
    "market":     "US",
    "currency":   "USD"    
}
# Send a POST request with JSON data
response = requests.post(url, json=data)
# Check the response status code
if response.status_code == 200:
    # Parse the JSON response and pretty-print it
    response_data = response.json()
    print("Response:")
    pprint(response_data)
else:
    print("Request failed with status code:", response.status_code)


Response:
{'asset_name': 'asset 1',
 'currency': 'USD',
 'market': 'US',
 'model_name': 'model 1'}


In [13]:
import requests

# Define the URL of your FastAPI application
id = 1
url = f"http://localhost:8000/{id}"

# Define the JSON data you want to send in the POST request
data = {
  'author': 'author 3_update'
}

# Send a PUT request with JSON data
response = requests.put(url, json=data)

# Check the response status code
if response.status_code == 200:
    # Parse the JSON response and pretty-print it
    response_data = response.json()
    print("Response:")
    print(response_data)
else:
    print("Request failed with status code:", response.status_code)


Response:
{'message': 'Book updated successfully'}
